<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_make_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#KcBERT 학습용 데이터셋을 제작하는 코드(로컬)
#제작할 데이터셋은 pretraining용 데이터셋(soynlp 학습용 데이터), finetuning용 데이터셋(next sentence prediction)

In [1]:
!pip install kss
!pip install datasets
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from kss import Kss
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.0/265.0 kB 18.

ModuleNotFoundError: No module named 'kss'

In [ ]:
#전처리된 파일 로드(BP_spacing 건너뛰기)
talk_save_path = '/content/drive/My Drive/talk_preprocess_result_short.xlsx'
qna_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'
all_contents_path = '/content/drive/My Drive/community_all_contents_preprocessed.csv'
all_comments_path = '/content/drive/My Drive/community_all_comments_preprocessed.csv'

df_talk = pd.read_excel(talk_save_path)
df_qna = pd.read_excel(qna_path)
df_all_contents = pd.read_csv(all_contents_path)
df_all_comments = pd.read_csv(all_comments_path)

In [ ]:
#직접 제작한 카카오톡 질문답변 데이터셋 로드
talk_finetune_path = '/content/drive/My Drive/talk_finetune_dataset.xlsx'

#Pretrain 데이터셋, 제공받은 모든 데이터 사용


In [ ]:
#kss로 리스트 안에 있는 여러 문장으로 이루어진 글을 문장 단위로 분리하는 함수
def seperate_to_sentences(corpus_list):
  new_corpus_list = []
  for corpus in corpus_list:
    new_corpus_list.append(kss.split_sentences(corpus))
  new_corpus_list = sum(lists for lists in new_corpus_list)
  return new_corpus_list

In [ ]:
corpus_list = ['딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?', '딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?', '딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?']
print(seperate_to_sentences(corpus_list))

In [ ]:
#kss로 리스트 안에 있는 여러 문장으로 이루어진 글을 문장 단위로 분리하는 함수
def separate_to_sentences_and_deduplicate(DataFrame, columns):
    sentences_list = []
    for column in columns: #데이터프레임의 각 열에 대해서 처리
        text_list = DataFrame[column].to_list()
        sentences_list.append(sum(kss.split_sentences(text) for text in text_list))
    return list(set(sum(lists for lists in sentences_list)))

In [ ]:
#데이터프레임의 열들을 리스트로 변환
talk_list = separate_to_sentences_and_deduplicate(df_talk, ['text'])

In [ ]:
qna_list = separate_to_sentences_and_deduplicate(df_qna, ['내용', '답변', '댓글'])

In [ ]:
all_contents_list = separate_to_sentences_and_deduplicate(df_all_contents, ['내용', '답변', '댓글'])

In [ ]:
all_comments_list = separate_to_sentences_and_deduplicate(df_all_comments, ['내용', '대댓글', '댓글'])

In [ ]:
#하나의 리스트로 결합
df_result = pd.DataFrame(sum(talk_list, qna_list, all_contents_list, all_comments_list), index = ['text']).T
print(df_result)

In [ ]:
#.xlsx 파일로 저장
pretrain_data_save_path = '/content/drive/My Drive/KcBERT_pretrain_dataset.xlsx'

df_result.to_excel(pretrain_data_save_path)

#Finetune 데이터셋


In [ ]:
#.xlsx 파일로 저장
finetune_data_save_path = '/content/drive/My Drive/KcBERT_finetune_dataset.xlsx'

df.to_excel(finetune_data_save_path)